In [0]:
from pyspark.sql.functions import lit
from datetime import date

initial_date = "2022-12-31"
spark.createDataFrame([(initial_date,)], ["last_processed_date"]) \
     .write.mode("overwrite") \
     .format("delta") \
     .saveAsTable("workspace.resume_project.processing_tracker")

In [0]:
from pyspark.sql.functions import col
from datetime import datetime, timedelta

tracker = spark.read.table("workspace.resume_project.processing_tracker")
last_date = tracker.collect()[0]["last_processed_date"]
next_date = (datetime.strptime(last_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

df = spark.read.table("workspace.resume_project.resume_master_source")
df_today = df.filter(col("upload_date") == next_date)

df_today.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.resume_project.resume_raw_data")

In [0]:
df_today.show(n=5)

+--------------------+------------------+------------+-------------------+
|              Resume|          Category|candidate_id|        upload_date|
+--------------------+------------------+------------+-------------------+
| enthusiastic com...|Operations Manager|      007018|2023-01-01 00:00:00|
|   State     Help...|             Other|      003532|2023-01-01 00:00:00|
|  <u> Micros </u>...|             Other|      007260|2023-01-01 00:00:00|
| </span> <span cl...|             Other|      009123|2023-01-01 00:00:00|
| </span> <span cl...|             Other|      017155|2023-01-01 00:00:00|
+--------------------+------------------+------------+-------------------+
only showing top 5 rows
